In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv("./data/train_dataset.csv")

data['is_test'] = False
evaluation = pd.read_csv("./data/evaluation_public.csv")
evaluation['is_test'] = True
sample = pd.read_csv("./data/sample_submission.csv")

all_data = pd.concat([data, evaluation]).reset_index(drop=True)

In [4]:
data[-4000:]['Label2'].describe()

count     3941.000000
mean     10873.759470
std       1933.837765
min       4825.330000
25%       9357.100000
50%      10697.480000
75%      12035.880000
max      24072.450000
Name: Label2, dtype: float64

In [5]:
data[-10000:]['Label2'].describe()

count     9941.000000
mean     11170.298525
std       2367.025648
min       4825.330000
25%       9333.600000
50%      10709.540000
75%      12336.400000
max      29788.960000
Name: Label2, dtype: float64

# 特征处理

In [6]:
all_data['date'] = pd.to_datetime(all_data['time'])
all_data['hour'] = all_data['date'].dt.hour
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['minute'] = all_data['date'].dt.minute
all_data['weekday'] = all_data['date'].dt.weekday
all_data['day'] = all_data['date'].dt.day
all_data['hour'] = all_data['date'].dt.hour
all_data['ts'] = all_data['hour']*3600 + all_data['minute']*60 + all_data['date'].dt.second

In [7]:
from tsfresh import extract_relevant_features
extract_relevant_features(all_data,all_data['Label1'], column_id="JS_SW", column_sort="date")

ValueError: The following ids are in the time series container but are missing in y: {nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 16.5779, 15.75, 15.875, 19.6087, 15.6875, 17.5, 20.625, 23.8048, 23.1866, 23.5539, 23.1318, 23.0702, 23.1798, 21.3125, 24.0779, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 21.438, 21.188, 21.1255, 21.0005, 22.938, 22.1255, 22.313, 22.188, 17.3125, 17.875, 23.0005, 23.438, 23.688, 19.375, 19.75, 23.563, 21.875, 21.1875, 21.75, 21.5625, 21.5, 22.5625, 22.625, 21.375, 22.875, 22.25, 23.375, 23.625, 23.875, 22.3125, 22.9375, 24.375, 24.125, 24.4375, 24.6875, 24.3125, 25.375, 25.8125, 24.875, 24.5, 24.0625, 25.25, 25.5625, 25.9375, 25.6875, 25.3125, 26.0625, 26.125, 26.3125, 26.1875, 26.5, 27.625, 27.8125, 27.875, 27.1875, 27.5, 28.125, 28.625, 28.0625, 29.625, 29.0625, 29.5, 29.5625, 29.8125, 30.1875, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 26.5005, 26.313, 26.938, 26.563, 27.2505, 27.0005, 27.313, 27.1255, 27.

In [ ]:
import gc
features = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'CS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL', 'N_QY_ORP','day','weekday','hour', 'ts']
features = [f for f in features if f not in ['time', 'Label1', 'Label2','Label1_log','Label2_log','CS_LL',
                                             'CS_NH3', 'JS_SW'
                                               # 'B_QY_ORP','JS_TN', 'CS_SW','MCCS_NH4','N_HYC_JS_DO','MCCS_NO3','JS_SW',
                                             ]]

labels = ['Label1', 'Label2']
train = all_data[~all_data['is_test']].copy(deep=True)
test = all_data[all_data['is_test']].copy(deep=True)
# train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)
test['is_train'] = False
train['is_train'] = True
data = pd.concat([train, test]).reset_index(drop=True)

for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data[f].drop_duplicates())
        data[f] = pd.qcut(data[f], q=int(q/10), labels=False, duplicates="drop")

del all_data
gc.collect()

In [ ]:
train = train.dropna(subset=['Label1', 'Label2']).reset_index(drop=True)

In [ ]:
train['Label2'][-1000].plot()

In [ ]:
train['Label1'][-2000:-1000].plot()

In [ ]:
predict_res = pd.read_csv("./res/merge_res.csv")

In [ ]:
predict_res['Label1'][1000:2000].plot()

In [ ]:
predict_res['Label2'][1000:2000].plot()

# 通过训练找出在训练集和测试集中差别很大的样本

In [ ]:
from sklearn.model_selection import train_test_split
train['is_test'] = 0
test['is_test'] = 1
data_v2 = pd.concat([train, test]).reset_index(drop=True)
for f in features:
    if f not in ['weekday','hour', 'ts']:
        q = len(data_v2[f].drop_duplicates())
        data_v2[f] = pd.qcut(data_v2[f], q=int(q/10), labels=False, duplicates="drop")

In [ ]:
import lightgbm as lgb
features_list = [ 'JS_NH3', 'CS_NH3', 'JS_TN', 'CS_TN', 'JS_LL', 'MCCS_NH4', 'MCCS_NO3', 'JS_COD', 'CS_COD', 'JS_SW', 'CS_SW', 'B_HYC_NH4', 'B_HYC_XD', 'B_HYC_MLSS', 'B_HYC_JS_DO', 'B_HYC_DO', 'B_CS_MQ_SSLL', 'B_QY_ORP', 'N_HYC_NH4', 'N_HYC_XD', 'N_HYC_MLSS', 'N_HYC_JS_DO', 'N_HYC_DO', 'N_CS_MQ_SSLL']

feature_score_dict = {}
for i in range(len(features_list)-1):
    for j in range(i+1, len(features_list)):
        features_filter = ['ts', 'weekday', features_list[i], features_list[j]]
        X_train, X_test, y_train, y_test  = train_test_split(data_v2[features_filter], data_v2['is_test'], test_size=0.2)
        model = lgb.LGBMClassifier(
                boosting="gbdt",
                max_depth=4,
                learning_rate=0.005,
                n_estimators=500,
                min_child_weight=1,
                min_data_in_leaf=60,
                subsample = 0.7,
                feature_fraction=0.4,
                bagging_seed=1,
                reg_alpha=0.11,
                reg_lambda=0.1,  # 此处不改了
                min_sum_hessian_in_leaf=0.01,
                random_state=1212
            )
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], eval_metric=['auc'],
                  early_stopping_rounds=20, verbose=200)
        feature_score_dict[f'{features_list[i]}_{features_list[j]}_score'] = model.best_score_['valid_1']['auc']

